The data is a of books on goodreads and i tried to see if I can predict the integer values of the rating for each book.

In [9]:
import numpy as np
from sklearn import tree
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import graphviz
import pydot
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

Want to classify rating of a book, on goodreads, within 5 classes(0 - 4)

In [2]:
file = open('./books.csv','r')
data_file = file.readlines()

Get the data into matrix form. The data points with more than 12 commas in them(only 5 of them) were disscarded for ease in vstacking

In [3]:
data = np.empty(shape=(0, len(data_file[0].split(','))))
#add each data point to the matrix having first split it into the features
for i in range(len(data_file)):
    point = data_file[i].split(',')
    if len(point) == 12:
        data = np.vstack((data, point))

Remove some unnecessary attributes(bookId,title,isbn,isb13) and then remove average rating after storing it on its own.

In [4]:
data0=data[:,2:] #remove title and bookid columns
data0 = data0[1:] #remove the string headings
data0 = np.delete(data0,2,1) #remove isbn
data0 = np.delete(data0,2,1) #remove isbn13

targets = data0[:,1] #get the average_rating to be the targets
data0 = np.delete(data0,1,1) #remove avarage_rating from data matrix

dates = data0[:,5] #get column of publication_dates
data0 = np.delete(data0, 5, 1) #remove publication_date column



THe publication data is going to be split in to three features: day, month, year:

In [5]:
day = np.empty(shape=0)
month = np.empty(shape=0)
year = np.empty(shape=0)

#extract each data point's day,month and year
for i in range(dates.shape[0]): #for each datapoint
    date = dates[i].split('/')
    day = np.append(day, int(date[0]))
    month = np.append(month, int(date[1]))
    year = np.append(year, int(date[2]))
    
#add the 3 arrays as features to data0
data0 = np.column_stack((data0,day))
data0 = np.column_stack((data0,month))
data0 = np.column_stack((data0,year))
    

In [6]:
print(data0[0])

['J.K. Rowling/Mary GrandPré' 'eng' '652' '2095690' '27591'
 'Scholastic Inc.\n' '9.0' '16.0' '2006.0']


The (author,language_code, publisher) columns must be encoded to numerical representation a sklearn's DTC only takes in numerical features

In [7]:
ordinal_encoder = OrdinalEncoder()
data1 = data0 #will be final processed version of the data

#encode the columns
enc_author = ordinal_encoder.fit_transform(data1[:,0].reshape(-1,1))
enc_lang = ordinal_encoder.fit_transform(data1[:,1].reshape(-1,1))
enc_publshr = ordinal_encoder.fit_transform(data[:,5].reshape(-1,1))

#replace the original columns with the encoded ones
data1[:,0] = enc_author.reshape(data1.shape[0])
data1[:,1] = enc_lang.reshape(data1.shape[0])
data1[:,5] = enc_publshr[1:].reshape(data1.shape[0])

Doing a PCA analysis on the data:

In [27]:
pca = PCA(n_components=2)
data2 = pca.fit_transform(data1)

Split the data into training and test data(4:1):

In [28]:
#this function converst a list of strings to floats then ints
def turnToInt(array):
    output = array
    for i in range(output.shape[0]):
        tmp0 = float(output[i])
        output[i] = int(tmp0)
    return output

training_data = data2[:int(8/10 * len(data1))]
training_targets = targets[:int(8/10 * len(data1))]

test_data = data2[len(training_data):]
test_targets = targets[len(training_data):]
#convert targets to 5 discreate classes
training_targets = turnToInt(training_targets)
test_targets = turnToInt(test_targets)


Train the training_data using Sklearns Decision tree classifier:

In [29]:
#train a descision tree classifier
clf = tree.DecisionTreeClassifier()
clf = clf.fit(training_data, training_targets) #train

Access it's perfomance using the test data:

In [30]:
#get the predictions of the trained model
predictions = clf.predict(test_data)

#create a confusion matrix
conf_mat = confusion_matrix(test_targets, predictions)

In [31]:
print(accuracy_score(test_targets,predictions))

0.5164044943820225


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf1 = KNeighborsClassifier()
clf1.fit(training_data, training_targets)

clf1_predictions = clf1.predict(test_data)
print("clf1 has ", accuracy_score(test_targets, clf1_predictions), " accuracy")

In [11]:
help(PCA)

Help on class PCA in module sklearn.decomposition._pca:

class PCA(sklearn.decomposition._base._BasePCA)
 |  Principal component analysis (PCA).
 |  
 |  Linear dimensionality reduction using Singular Value Decomposition of the
 |  data to project it to a lower dimensional space. The input data is centered
 |  but not scaled for each feature before applying the SVD.
 |  
 |  It uses the LAPACK implementation of the full SVD or a randomized truncated
 |  SVD by the method of Halko et al. 2009, depending on the shape of the input
 |  data and the number of components to extract.
 |  
 |  It can also use the scipy.sparse.linalg ARPACK implementation of the
 |  truncated SVD.
 |  
 |  Notice that this class does not support sparse input. See
 |  :class:`TruncatedSVD` for an alternative with sparse data.
 |  
 |  Read more in the :ref:`User Guide <PCA>`.
 |  
 |  Parameters
 |  ----------
 |  n_components : int, float, None or str
 |      Number of components to keep.
 |      if n_component